In [1]:
import random
import string
import numpy as np
import matplotlib.pyplot as plt

from skimage.io import imshow
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model, Sequential
from keras.layers import Input, Activation
from keras.layers.core import Dropout, Lambda, Dense, Flatten
from keras.layers.convolutional import Conv2D, Conv2DTranspose, Convolution2D
from keras.layers.pooling import MaxPooling2D,MaxPool2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras.optimizers import Adadelta

import tensorflow as tf
from sklearn.model_selection import train_test_split

/home/claudio/Documents/virtual_envs/deep_learning/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/claudio/Documents/virtual_envs/deep_learning/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
def captcha_acc(real, pred, seq_len=4, char_set=36):
    y_real = tf.reshape(real, [seq_len,char_set])
    y_pred = tf.reshape(pred, [seq_len,char_set])
    return 1 - tf.reduce_mean(tf.reduce_sum(tf.reduce_sum(tf.abs(y_real-y_pred,0),0)))

In [ ]:
def captcha_loss(real, pred, seq_len=4, char_set=36):
    y_real = tf.reshape(real, [seq_len,char_set])
    y_pred = tf.reshape(pred, [seq_len,char_set])
    return tf.reduce_mean(tf.reduce_sum(tf.reduce_sum(tf.abs(y_real-y_pred,0),0)))

In [15]:
x = np.random.rand(6,2,3)

In [16]:
x

array([[[0.40878586, 0.97309058, 0.40620048],
        [0.13911216, 0.28321539, 0.56666692]],

       [[0.1427571 , 0.96046582, 0.97762509],
        [0.0215152 , 0.42530115, 0.57221198]],

       [[0.24848314, 0.27054269, 0.54710951],
        [0.86198634, 0.2259323 , 0.15844201]],

       [[0.25051725, 0.50630304, 0.70236594],
        [0.61382688, 0.38796361, 0.19435676]],

       [[0.21850809, 0.16526575, 0.12251872],
        [0.80350454, 0.42496343, 0.00299896]],

       [[0.2242832 , 0.47782686, 0.2491036 ],
        [0.18977231, 0.74695987, 0.7445317 ]]])

In [ ]:
K.one_hot()

In [22]:
np.argmax(x,axis=-1)

array([[1, 2],
       [2, 2],
       [2, 0],
       [2, 0],
       [0, 0],
       [1, 1]])

In [2]:
def captcha_loss(real, pred):
    y_real = K.argmax(K.reshape(real, [4,36]),0)
    y_pred = K.argmax(K.reshape(pred, [4,36]),0)
    return K.mean(K.sum(K.sum(K.abs(y_real-y_pred),0)))

In [5]:
def captcha_acc(real, pred):
    y_real = K.argmax(K.reshape(real, [4,36]),0)
    y_pred = K.argmax(K.reshape(pred, [4,36]),0)
    return 1 - K.mean(K.sum(K.sum(K.abs(y_real-y_pred),0)))

In [4]:
X_train = np.load('data/train_inputs.npy')
y_train = np.load('data/train_targets.npy')

In [6]:
height = 60
width = 160
num_classes = 36
word_len = 4
learning_rate = 0.1

In [7]:
input_tensor = Input(shape=(height, width, 3))

x = input_tensor

x = Conv2D(32, 3, padding='same', activation='relu')(x)
x = Conv2D(32, 3, padding='same', activation='relu')(x)
x = Activation("relu")(x)
x = MaxPool2D((2, 2))(x)

x = Conv2D(64, 3, padding='same', activation='relu')(x)
x = Conv2D(64, 3, padding='same', activation='relu')(x)
x = Activation("relu")(x)
x = MaxPool2D((2, 2))(x)

x = Conv2D(128, 3, padding='same', activation='relu')(x)
x = Conv2D(128, 3, padding='same',activation='relu')(x)
x = Activation("relu")(x)
x = MaxPool2D((2, 2))(x)

x = Conv2D(256, 3, padding='same', activation='relu')(x)
x = Conv2D(256, 3, padding='same',activation='relu')(x)
x = Activation("relu")(x)
x = MaxPool2D((2, 2))(x)

x = Flatten()(x)
x = Dropout(0.25)(x)

x = [Dense(num_classes, activation='softmax', name='c%d'%(i+1))(x) for i in range(word_len)]
output = concatenate(x)

In [8]:
model = Model(inputs=input_tensor, outputs=output)

opt = Adadelta(lr=learning_rate)

model.compile(loss = captcha_loss, optimizer=opt, metrics=['accuracy'])

model.summary()

ValueError: Tensor conversion requested dtype int64 for Tensor with dtype float32: 'Tensor("concatenate_1_sample_weights:0", shape=(?,), dtype=float32)'

In [ ]:
K.redu

In [ ]:
earlystopper = EarlyStopping(patience=5, verbose=1)

checkpointer = ModelCheckpoint(filepath="weights.{epoch:02d}--{val_loss:.2f}-{val_acc:.4f}.hdf5", 
                               verbose=1, save_weights_only=True)

model.fit(X_train, y_train, epochs= 15, validation_split=0.1, callbacks=[earlystopper])

In [ ]:
X_test = np.load('data/test_inputs.npy')
y_test = np.load('data/test_targets.npy')

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
imshow(X_test[0])
imshow(X_test[6]);

In [ ]:
y_predicted = model.predict(X_test[0:7,:,:])

In [ ]:
max_ix = np.argmax(y_predicted[6].reshape((4,36)), axis=1)

In [ ]:
char_set = list(string.digits + string.ascii_letters)

In [ ]:
string_ = ''
for i in max_ix:
    string_ += char_set[i]

print(string_)

In [ ]:
def captcha_accuracy(y_test, y_pred)
    y_test = y_test.reshape((4,36))
    y_pred = y_pred.reshape((4,36))
    
    y_test - y_pred

In [ ]:
a = np.array([[0,0,0,1]
              [0,1,0,0]])

b = np.array([[0,0,0,1]
              [0,1,0,0]])

In [ ]:
a - b